In [1]:
import numpy as np
import cv2 
import socket
import serial

In [5]:


# Configurar la comunicación serial con Arduino
arduino = serial.Serial('/dev/ttyACM0', 9600)  # Asegúrate de usar el puerto correcto

# Configurar el servidor de sockets
HOST = '192.168.31.47'  # Escuchar en todas las interfaces de red
PORT = 8080      # Puerto en el que el servidor escuchará

def handle_client_connection(client_socket):
    with client_socket:
        print('Conexión establecida')
        while True:
            data = client_socket.recv(1024)
            if not data:
                break
            print("Datos recibidos:", data.decode())
            arduino.write(data)  # Enviar los datos recibidos a Arduino
            response = arduino.readline().decode().strip()
            client_socket.sendall(response.encode())  # Enviar la respuesta de Arduino de vuelta al cliente

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as server_socket:
    server_socket.bind((HOST, PORT))
    server_socket.listen()
    print("Servidor de sockets escuchando en el puerto", PORT)
    while True:
        client_socket, addr = server_socket.accept()
        print('Conexión establecida por', addr)
        handle_client_connection(client_socket)

SerialException: [Errno 16] could not open port /dev/ttyACM0: [Errno 16] Device or resource busy: '/dev/ttyACM0'

In [ ]:
# Configuración del cliente
HOST = '192.168.31.133'  # Dirección IP del servidor App Inventor
PORT = 8080              # Puerto del servidor

In [ ]:
def enviar_datos(mensaje):
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as client_socket:
        client_socket.connect((HOST, PORT))
        client_socket.sendall(mensaje.encode())
        print(f"Enviado: {mensaje.encode()}")



In [ ]:
def extraerCentro(frame):
    dimH = frame.shape[1]
    dimV = frame.shape[0]
    rango = 150
    
    # Aplicamos un filtro negro:
    imagenInvert = abs(250 - frame)
    imagenInvert[imagenInvert < rango] = 0
    imagenInvert[imagenInvert >= rango] = 250
    
    # Establecemos la mitad vertical
    mitadV = dimV // 2
    pistaInicio = 0
    pistaFin = dimH

    # Encontrar los bordes de la pista
    for j in range(dimH):
        pixel = imagenInvert[mitadV][j]
        if pixel == 250 and pistaInicio == 0:
            pistaInicio = j
        if pixel == 0 and pistaInicio != 0:
            pistaFin = j-1
            break

    # Retorno las coordenadas del centro (altura, ancho)
    puntoH = (pistaFin + pistaInicio) // 2
    #print("Coordenadas horizontales : ", (puntoH))
    return (mitadV, puntoH)

In [ ]:
def dibujarCruz(frame, puntoH, puntoV,mensaje):
    grosor = 5
    s = frame.copy()
    # linea vertical
    s[:,puntoH - grosor :puntoH + grosor ,:] = 255
    s[puntoV - grosor :puntoV + grosor,: ,:] = 255
    
    ## Dibujar el rango permitido para mantenerse en el centro: 
    # Escribir texto sobre el frame
    cv2.putText(s, 
                mensaje, 
                (50, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 
                1, 
                (255, 255, 255), 
                2, 
                cv2.LINE_AA)
    return s

In [ ]:
def decision(puntoActual, d):
    f,c = d
    columnas = c//2
    margen = 20
    minimo = columnas - margen
    maximo = columnas + margen
    retorno = ""
    if(puntoActual > maximo) : 
        # Gira a la derecha :
        retorno = ("d")
    elif (puntoActual < minimo ):
        # Gira a la izquierda : 
        retorno = ("a")
    else:
        # Está en el centro 
        retorno = ("w")
    return (retorno,abs(puntoActual - columnas ))

In [ ]:
def detectar_centro_pista():
    ip = "192.168.31.133:8080"
    cam = cv2.VideoCapture(f"rtsp://{ip}/h264_ulaw.sdp")
    
    if not cam.isOpened():
        print("Error: No se puede abrir el flujo de video")
        return
    
    while True:
        ret, frame = cam.read()
        if not ret:
            print("Error: No se puede leer el cuadro del video")
            break

        # Convertir a escala de grises
        gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        
        # Extraer el centro
        puntoV, puntoH = extraerCentro(gris)
        
        # Determino la decision a tomar : 
        
        mensaje,diferencia = decision(puntoH, gris.shape)
        
        # Dibujar la cruz en el centro
        fin = dibujarCruz(frame, puntoH, puntoV,mensaje + f"{diferencia} pixeles")

        # Mostrar el video con la pista y el centro detectado
        cv2.imshow('Video con Centro de Pista', fin)
        
        # Enviar mensaje al app
        enviar_datos(mensaje)
        
        # Terminar con 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cam.release()
    cv2.destroyAllWindows()

In [ ]:
detectar_centro_pista()

Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a
Enviado: a

In [ ]:
# import cv2

# # Reemplaza esta URL con la URL proporcionada por la app IP Webcam
# url = "http://192.168.31.133:8080/video"

# # Abre el stream de video
# cap = cv2.VideoCapture(url)

# if not cap.isOpened():
#     print("Error al abrir el stream de video")
#     exit()

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("Error al recibir el frame")
#         break

#     # Muestra el frame capturado
#     cv2.imshow('IP Webcam', frame)

#     # Salir si se presiona la tecla 'q'
#     if cv2.waitKey(1) & 0xFF == ord('q'):
#         break

# # Liberar los recursos
# cap.release()
# cv2.destroyAllWindows()


In [ ]:
import socket
s = socket.socket()
s.bind(("192.168.31.47", 8080))
s.listen(10)
print("loading...")
connected = True

while True: 
    connected = True
    (client, addr) = s.accept()
    print("client connected: ", addr)
    while connected:
        data = client.recv(1024)
        if data:
            print(data)
        else:
            connected = False

    print("client disconnected")
    client.close()
    s.close()
    print("game over")
    s.sendall().__doc__

loading...


In [ ]:
pip install serial

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 119.2 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.3/491.3 kB 2.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 757.7/757.7 kB 3.9 MB/s eta 0:00:00:00:0100:01
Note: you may need to restart the kernel to use updated packages.
